## AgentExecutor (ReAct Agent — แบบเก่า/Legacy)

เอเจนต์จะวนลูป Reason → Act(tool) → Observe → อัตโนมัติ และเลือกใช้ search หรือ calculate_math_tool ได้เองหลายรอบ แต่แนวนี้ถูกจัดเป็น legacy และมีคำแนะนำให้ย้ายไป LangGraph หากงานซับซ้อนขึ้น

### Build Tool Function

In [ ]:
from langchain.agents import Tool

def calculate_math_tool(expression: str):
    """Calculate mathematical expressions safely"""
    try:
        allowed_chars = set('0123456789+-*/(). ')
        if not all(c in allowed_chars for c in expression):
            return "ข้อผิดพลาด: นิพจน์ไม่ถูกต้อง"
        return str(eval(expression))
    except Exception as e:
        return f"ข้อผิดพลาดในการคำนวณ: {str(e)}"

calculate_tool = Tool(
    name="calculator",
    func=calculate_math_tool,
    description="Use this to calculate mathematical expressions. Input like '2 + 3 * 4'."
)

calculate_tool.invoke("1+2")

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

search_tool = Tool(
    name="search",
    description="Useful for when you need more information from an online search",
    func=serper.run
)

### Create Agent

- create_tool_calling_agent -> Agent แบบ function/tool calling (เช่น Gemini, GPT-4)
- create_openai_functions_agent -> สำหรับ OpenAI function calling โดยเฉพาะ
- create_structured_chat_agent -> แบบ structured output (ใช้ JSON schema)
- create_react_agent -> Agent แบบ ReAct (reasoning + action)
- create_self_ask_with_search_agent -> Agent ที่ถามซ้อนและค้นข้อมูล
- create_xml_agent -> Agent ที่รับ/ส่งข้อมูลในรูปแบบ XML
- create_multi_function_agent -> Agent ที่เรียกหลาย tool พร้อมกัน (multi-action agent)

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.tools import Tool
# from langgraph.prebuilt import create_react_agent
# from langchain_community.chat_models import ChatOllama  # ✅ ใช้ Ollama

# # หากใช้ Ollama แบบ local ต้องติดตั้ง langchain-community และรัน ollama ที่มีโมเดล gemma3

# # หากมี search_tool เพิ่มตรงนี้ด้วย
# TOOLS: list[Tool] = [calculate_tool, ] 

# # ✅ Prompt พร้อม input_variables
# prompt = ChatPromptTemplate.from_messages(
#     messages=[
#         (
#             "system",
#             "You are a helpful assistant. You have access to these tools:\n{tools}\n\n"
#             "When you need a tool, choose from: {tool_names}.\n"
#             "You MUST follow the EXACT ReAct format:\n"
#             "Thought: I need to think about what to do\n"
#             "Action: tool_name\n"
#             "Action Input: input_for_tool\n"
#             "Observation: result_from_tool\n"
#             "Thought: I can now answer the question\n"
#             "Final Answer: final response\n\n"
#             "Always use this exact format with proper line breaks.",
#         ),
#         ("human", "{input}\n\n{agent_scratchpad}"),
#     ],
# )

# # ✅ LLM Model
# model = ChatOllama(model="gpt-oss", temperature=0)

# # ✅ Create ReAct Agent
# agent = create_react_agent(
#     model=model,
#     tools=TOOLS,
#     prompt=prompt,
# )

# # ✅ Run Agent
# response = agent.invoke(
#     {"messages": [{"role": "user", "content": "คำนวณ (6+2)*3 พร้อมคำอธิบาย"}]}
# )
# print(response)

In [ ]:
tools = [calculate_tool, search_tool]

In [ ]:

from langchain_ollama import ChatOllama

llm = ChatOllama(model="gpt-oss")

llm_with_tools = llm.bind_tools(tools)

print(llm_with_tools.invoke(input="คำนวณ (6+2)*3 พร้อมคำอธิบาย"))


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage
from langchain_core.tools import Tool

# ✅ เครื่องมือ (tool)
def calculate(expression: str) -> str:
    try:
        result = eval(expression)
        return f"ผลลัพธ์ของ {expression} คือ {result}"
    except Exception as e:
        return f"เกิดข้อผิดพลาด: {str(e)}"

calculate_tool = Tool(
    name="Calculator",
    func=calculate,
    description="ใช้สำหรับคำนวณนิพจน์คณิตศาสตร์ เช่น (6+2)*3"
)

TOOLS = [calculate_tool]

# ✅ Prompt Template
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. You have access to these tools:\n{tools}\n\n"
        "When you need a tool, choose from: {tool_names}.\n"
        "You MUST follow the EXACT ReAct format:\n"
        "Thought: I need to think about what to do\n"
        "Action: tool_name\n"
        "Action Input: input_for_tool\n"
        "Observation: result_from_tool\n"
        "Thought: I can now answer the question\n"
        "Final Answer: final response\n\n"
        "Always use this exact format with proper line breaks."
    ),
    ("human", "{input}\n\n{agent_scratchpad}"),
])

# ✅ สร้าง LLM
llm = ChatOllama(model="llama3.1", temperature=0)

# ✅ ผูก prompt กับ LLM
chain: Runnable = prompt | llm

# ✅ ทดสอบรัน
question = "คำนวณ (6+2)*3 พร้อมคำอธิบาย"
response = chain.invoke({
    "input": question,
    "tools": "\n".join([f"- {tool.name}: {tool.description}" for tool in TOOLS]),
    "tool_names": ", ".join([tool.name for tool in TOOLS]),
    "agent_scratchpad": ""  # ถ้ามี multi-turn memory ค่อยใส่
})

print(response.content)
